In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline
import nibabel as nib
import numpy.ma as ma
from numpy import savetxt
import os


In [2]:
#input variables and path defs
workDir = '/host/percy/local_raid/hans/amyg/struct/'
outDir = str(workDir+'outputs/')
volDir = str(workDir+'volumes/')
regDir = str(workDir+'registration/')

In [ ]:
def numexpr_based_with_slicing(A,B):
    a0 = A[...,0]
    a1 = A[...,1]
    b0 = B[...,0]
    b1 = B[...,1]
    return ne.evaluate('sqrt((a0-b0)**2 + (a1-b1)**2)')

def run_euc(list_a,list_b):
    return np.array([[ numexpr_based_with_slicing(i,j) for j in list_b] for i in list_a])


side='left'
side2='L'
for i in [1,4,5,6,7,9,10,11,12,13]:
    if i <10:
        sub=str('PNC00'+str(i))
    else:
        sub=str('PNC0'+str(i))
    print(sub)
    ses='01'

    # load mask to clean off all background data from feature data
    # pick different masks depending on the cropping size
    maskNif = nib.load(outDir+'amyg_mask_'+sub+'_'+ses+'_'+side+'_roi.nii.gz')
    mask = np.array(maskNif.dataobj)

    #turn mask into a list
    maskBin = np.where(mask == 1, 0, 1)
    maskBin = maskBin.tolist()
    #Load UMAP embeddings
    nn=15
    md=0.1
    path = str(outDir +sub+"_"+ses+'_'+side2+'_embedding_1sd.csv')
    embedding=np.loadtxt(path,delimiter=',')
    print(len(embedding))
    #create matrix with corresponding coordinates for each U1/2 values of cropped 1 in every 5x5x5
    mapNif1 = nib.load(regDir+sub+'_'+ses+'_'+side+'_amyg_u1_MNI152Space_1sd.nii.gz')
    mapNif2 = nib.load(regDir+sub+'_'+ses+'_'+side+'_amyg_u2_MNI152Space_1sd.nii.gz')
    #mapNif1 = nib.load(outDir+'amyg_'+sub+'_'+ses+'_left_u1_1sd.nii.gz')
    mapU1 = np.array(mapNif1.dataobj)
    #mapNif2 = nib.load(outDir+'amyg_'+sub+'_'+ses+'_left_u2_1sd.nii.gz')
    mapU2 = np.array(mapNif2.dataobj)
    print(len(mapU2))
    coords=np.zeros((len(embedding)+1,5))
    num=0
    for i in range(len(mapU1[:,1,1])):
        for j in range(len(mapU1[1,:,1])):
            for k in range(len(mapU1[1,1,:])):
                if mapU1[i,j,k] !=0:
                    coords[num,0]=i
                    coords[num,1]=j
                    coords[num,2]=k
                    coords[num,3]=mapU1[i,j,k]
                    coords[num,4]=mapU2[i,j,k]
                    num=num+1

    import numexpr as ne
    list_a = coords[:,0:3]
    U1 = coords[:,3]

    #print(list_a)

    D = run_euc(list_a, list_a)

    dM=D
    Dsort= np.argsort(D, axis=1)

    from brainspace.null_models import SampledSurrogateMaps
    X=coords[:,0]
    Y=coords[:,1]
    Z=coords[:,2]
    U2 = coords[:,4]
    ssm = SampledSurrogateMaps(ns=1000, knn=500, random_state=0)
    ssm.fit(D,Dsort)

    from scipy.stats import spearmanr
    U2=coords[:,4]
    X_surrogates = ssm.randomize(X, n_rep=1000)
    Y_surrogates = ssm.randomize(Y, n_rep=1000)
    Z_surrogates = ssm.randomize(Z, n_rep=1000)

    #make plot
    fig, axs = plt.subplots(1, 3, figsize=(9, 3.5))

    feats = {'I-S': X, 'P-A': Y, 'M-L':Z}
    rand = {'I-S': X_surrogates, 'P-A': Y_surrogates, 'M-L': Z_surrogates}
    pv_obs=1
    for k, (fn, data) in enumerate(rand.items()):
        r_obs,pv_obs = spearmanr(feats[fn], U1, nan_policy='omit')

        # Compute perm pval
        r_rand = np.asarray([spearmanr(U1, d)[0] for d in data])
        pv_rand = np.mean(np.abs(r_rand) >= np.abs(r_obs))

        # Plot null dist
        weights = np.ones_like(r_rand) / (len(r_rand))
        axs[k].hist(r_rand, bins=25, density=False,weights=weights, alpha=0.5, color=(.8, .8, .8))
        axs[k].axvline(r_obs, lw=2, ls='--', color='k')
        axs[k].set_xlabel(f'Correlation with {fn}')
        if k == 0:
            axs[k].set_ylabel('Density')

        print(f'{fn.capitalize()}:\n Obs      : {pv_obs:.5e}\n '
              f'Variogram: {pv_rand:.5e}\n')
        #print(r_rand[:10])
    fig.tight_layout()
    #plt.show()
    fig.savefig(str("/home/bic/hauer/structural/variogram_"+side+"_u1_"+str(sub)))

    #make plot

    fig, axs = plt.subplots(1, 3, figsize=(9, 3.5))

    feats = {'I-S': X, 'P-A': Y, 'M-L':Z}
    rand = {'I-S': X_surrogates, 'P-A': Y_surrogates, 'M-L': Z_surrogates}
    pv_obs=1
    for k, (fn, data) in enumerate(rand.items()):
        r_obs,pv_obs = spearmanr(feats[fn], U2, nan_policy='omit')

        # Compute perm pval
        r_rand = np.asarray([spearmanr(U2, d)[0] for d in data])
        pv_rand = np.mean(np.abs(r_rand) >= np.abs(r_obs))

        # Plot null dist
        weights = np.ones_like(r_rand) / (len(r_rand))
        axs[k].hist(r_rand, bins=25, density=False,weights=weights, alpha=0.5, color=(.8, .8, .8))
        axs[k].axvline(r_obs, lw=2, ls='--', color='k')
        axs[k].set_xlabel(f'Correlation with {fn}')
        if k == 0:
            axs[k].set_ylabel('Density')

        print(f'{fn.capitalize()}:\n Obs      : {pv_obs:.5e}\n '
              f'Variogram: {pv_rand:.5e}\n')
        #print(r_rand[:10])
    fig.tight_layout()
    #plt.show()
    fig.savefig(str("/home/bic/hauer/structural/variogram_"+side+"_u2_"+str(sub)))